# K-fold

2023.07.21 2027i
lets get init

In [102]:
import pandas as pd

bankdf = pd.read_csv('UniversalBank.csv')
bankdf

,ID,Age,Experience,Income,ZIPCode,Family,CCAvg,Education,Mortgage,PersonalLoan,SecuritiesAccount,CDAccount,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,29,3,40,92697,1,1.9,3,0,0,0,0,1,0
4996,4997,30,4,15,92037,4,0.4,1,85,0,0,0,1,0
4997,4998,63,39,24,93023,2,0.3,3,0,0,0,0,0,0
4998,4999,65,40,49,90034,3,0.5,2,0,0,0,0,1,0


In [103]:
x = bankdf.drop(['ID','ZIPCode','PersonalLoan'],axis=1)
y = bankdf['PersonalLoan']

In [104]:
from sklearn.model_selection import train_test_split

xtrain,xtest,ytrain,ytest = train_test_split(x,y,stratify=y)

In [105]:
xtrain

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,SecuritiesAccount,CDAccount,Online,CreditCard
3998,34,10,41,1,1.33,1,0,0,0,0,1
3817,65,40,140,1,0.90,1,0,0,0,0,0
4150,46,20,72,2,1.70,2,75,0,0,1,0
2303,47,21,89,2,0.80,3,0,0,0,1,1
1089,53,29,94,4,1.00,2,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
1022,27,3,118,1,3.30,2,0,0,0,1,0
620,33,8,115,4,2.90,2,0,0,0,1,0
4865,50,24,133,4,1.40,2,342,0,0,0,1
3192,65,39,35,1,0.50,3,0,0,0,0,0


In [106]:
xtest

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,SecuritiesAccount,CDAccount,Online,CreditCard
2852,54,29,183,1,8.1,1,0,0,0,1,1
1545,55,29,131,2,2.7,1,0,1,0,0,0
2507,59,34,60,4,2.1,1,234,0,0,1,1
411,60,36,54,4,2.3,3,0,1,0,0,0
3096,43,18,179,3,1.2,1,0,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...
1486,35,9,141,2,4.5,2,0,0,0,0,0
950,32,6,112,4,1.8,2,0,0,0,1,0
1659,33,7,139,1,4.0,3,106,0,1,1,1
821,39,13,33,4,1.5,3,0,0,0,0,0


In [107]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

logistic = LogisticRegression(solver='liblinear',
                              penalty='l2',
                              C=0.001,)
tree = DecisionTreeClassifier(max_depth=None,
                              criterion='entropy',)
knn = KNeighborsClassifier(n_neighbors=1,
                           p=2,
                           metric='minkowski')

In [108]:
from sklearn.ensemble import VotingClassifier
voting_estimators = [('logistic',logistic),('tree',tree),('knn',knn)]
voting = VotingClassifier(estimators=voting_estimators,
                          voting='soft')

In [109]:
from sklearn.model_selection import cross_val_score

clf_labels = ['LogisticRegression','Decision tree','KNN','Majority voting']
all_clf = [logistic,tree,knn,voting]

for clf,label in zip(all_clf,clf_labels):
    scores = cross_val_score(estimator=clf,X=xtrain,y=ytrain,cv=10,scoring='roc_auc')
    print("ROC AUC : %0.3f (+/- %0.3f) [%s]"%(scores.mean(),scores.std(),label))

ROC AUC : 0.926 (+/- 0.025) [LogisticRegression]
ROC AUC : 0.948 (+/- 0.024) [Decision tree]
ROC AUC : 0.696 (+/- 0.046) [KNN]
ROC AUC : 0.970 (+/- 0.013) [Majority voting]


In [110]:
from sklearn.model_selection import GridSearchCV
params = {'logistic__C' : [0.001,0.1,100],
          'tree__max_depth' : [1,3,5],
          'knn__n_neighbors' : [1,3,5]}

grid = GridSearchCV(estimator=voting,
                    param_grid=params,
                    cv=10,
                    scoring='roc_auc',)

grid.fit(xtrain,ytrain)

for r, _ in enumerate(grid.cv_results_['mean_test_score']):
    print("%0.3f +/- %0.3f %r" % (grid.cv_results_['mean_test_score'][r],
                                  grid.cv_results_['std_test_score'][r]/2,
                                  grid.cv_results_['params'][r]))
    

print('best param : %s' %grid.best_params_)
print('AUC : %.3f'%grid.best_score_)

0.928 +/- 0.012 {'knn__n_neighbors': 1, 'logistic__C': 0.001, 'tree__max_depth': 1}
0.971 +/- 0.007 {'knn__n_neighbors': 1, 'logistic__C': 0.001, 'tree__max_depth': 3}
0.976 +/- 0.006 {'knn__n_neighbors': 1, 'logistic__C': 0.001, 'tree__max_depth': 5}
0.939 +/- 0.012 {'knn__n_neighbors': 1, 'logistic__C': 0.1, 'tree__max_depth': 1}
0.979 +/- 0.005 {'knn__n_neighbors': 1, 'logistic__C': 0.1, 'tree__max_depth': 3}
0.983 +/- 0.004 {'knn__n_neighbors': 1, 'logistic__C': 0.1, 'tree__max_depth': 5}
0.943 +/- 0.011 {'knn__n_neighbors': 1, 'logistic__C': 100, 'tree__max_depth': 1}
0.981 +/- 0.004 {'knn__n_neighbors': 1, 'logistic__C': 100, 'tree__max_depth': 3}
0.985 +/- 0.004 {'knn__n_neighbors': 1, 'logistic__C': 100, 'tree__max_depth': 5}
0.933 +/- 0.011 {'knn__n_neighbors': 3, 'logistic__C': 0.001, 'tree__max_depth': 1}
0.978 +/- 0.006 {'knn__n_neighbors': 3, 'logistic__C': 0.001, 'tree__max_depth': 3}
0.983 +/- 0.005 {'knn__n_neighbors': 3, 'logistic__C': 0.001, 'tree__max_depth': 5}
0.94

# Bagging

In [111]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(max_depth=None,
                              criterion='entropy',)

In [112]:
from sklearn.ensemble import BaggingClassifier

bagging = BaggingClassifier(base_estimator=tree,
                            n_estimators=500,
                            max_samples=1.0,
                            max_features=1.0,
                            bootstrap=True,
                            bootstrap_features=False,
                            n_jobs=1,)

In [113]:
from sklearn.model_selection import cross_val_score

clf_labels = ['Decition tree','Bagging'] #decition tree VS Bagging. Lets see whats better!
all_clf = [tree,bagging]

for clf,label in zip(all_clf,clf_labels):
    scores = cross_val_score(estimator=clf,X=xtrain,y=ytrain,cv=10,scoring='roc_auc')
    print("ROC AUC : %0.3f (+/- %0.3f) [%s]"%(scores.mean(),scores.std(),label))

ROC AUC : 0.946 (+/- 0.023) [Decition tree]


/Users/owo/anaconda3/envs/torchenv/lib/python3.11/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/Users/owo/anaconda3/envs/torchenv/lib/python3.11/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/Users/owo/anaconda3/envs/torchenv/lib/python3.11/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/Users/owo/anaconda3/envs/torchenv/lib/python3.11/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/Users/owo/anaconda3/envs/torchenv/lib/python3.11/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `e

ROC AUC : 0.998 (+/- 0.001) [Bagging]


# Boosting

In [114]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(max_depth=1,
                              criterion='entropy',)

In [115]:
from sklearn.ensemble import AdaBoostClassifier

adaboost = AdaBoostClassifier(base_estimator=tree,
                              n_estimators=500,
                              learning_rate=0.1)

In [116]:
from sklearn.model_selection import cross_val_score


clf_labels = ['Decition tree','Ada boost'] #decition tree VS Ada boots. Lets see whats better!
all_clf = [tree,adaboost]

for clf,label in zip(all_clf,clf_labels):
    scores = cross_val_score(estimator=clf,X=xtrain,y=ytrain,cv=10,scoring='roc_auc')
    print("ROC AUC : %0.3f (+/- %0.3f) [%s]"%(scores.mean(),scores.std(),label))

ROC AUC : 0.875 (+/- 0.025) [Decition tree]


/Users/owo/anaconda3/envs/torchenv/lib/python3.11/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/Users/owo/anaconda3/envs/torchenv/lib/python3.11/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/Users/owo/anaconda3/envs/torchenv/lib/python3.11/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/Users/owo/anaconda3/envs/torchenv/lib/python3.11/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/Users/owo/anaconda3/envs/torchenv/lib/python3.11/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `e

ROC AUC : 0.980 (+/- 0.012) [Ada boost]


# Pipeline

In [117]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(max_depth=None,
                              criterion='gini',)
tree.fit(xtrain,ytrain)

DecisionTreeClassifier()

In [119]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier

#pipe_tree = make_pipeline(StandardScaler(), DecisionTreeClassifier(), PCA(n_components=10))
pipe_tree = make_pipeline(DecisionTreeClassifier())

param_range1 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
param_range2 = [10, 20, 30, 40, 50, 60, 70, 80, 90]

param_grid = [{'decisiontreeclassifier__max_depth': param_range1,
               'decisiontreeclassifier__min_samples_leaf': param_range2}]

gs = GridSearchCV(estimator=pipe_tree,
                  param_grid=param_grid,
                  scoring='accuracy',
                  cv=10,
                  n_jobs=-1)

gs = gs.fit(xtrain, ytrain)

print(gs.best_score_)
print(gs.best_params_)


0.9821333333333333
{'decisiontreeclassifier__max_depth': 5, 'decisiontreeclassifier__min_samples_leaf': 30}


In [121]:
from sklearn.metrics import confusion_matrix,classification_report

best_tree = gs.best_estimator_
best_tree.fit(xtrain,ytrain)

ypred = best_tree.predict(xtest)

print(classification_report(ytest,ypred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1130
           1       1.00      0.85      0.92       120

    accuracy                           0.99      1250
   macro avg       0.99      0.93      0.96      1250
weighted avg       0.99      0.99      0.99      1250

